## Surface Determination Workflow  

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import h5py
import sys
from tomo_encoders import Patches, DataFile
import tensorflow as tf
import time, glob
import open3d as o3d

sys.path.append('/data02/MyArchive/aisteer_3Dencoders/TomoEncoders/scratchpad/surface_determination/trainer')
from tomo_encoders.neural_nets.surface_segmenter import SurfaceSegmenter
from tomo_encoders.misc.feature_maps_vis import view_midplanes
from tomo_encoders.misc.voxel_processing import normalize_volume_gpu
from tomo_encoders.misc.viewer import view_midplanes

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def get_slices(o, c):
    
    points = []
    for ii in range(2):
        for jj in range(2):
            for kk in range(2):
                points.append([ii*c, jj*c, kk*c])
                
    points = np.asarray(points) + o
    
    
    
    return [tuple([slice(int(point[ii]), int(point[ii] + c)) for ii in range(3)]) for point in points], points

def split_cube(vol, point, target_width):
    slice_set, points = get_slices(point, target_width)
    
    vols = []
    
    for i, s in enumerate(slice_set):
        
        vols.append(vol[s])
    vols = np.asarray(vols)
    
    mask = np.std(vols, axis = (1,2,3)) > 0.1
    idx_sel = np.where(mask)[0]
    vols = vols[idx_sel,...]
    points = points[idx_sel,...]
    
    return vols, points

In [3]:
vol = np.zeros((512,512,512))
vol[100:-100, 100:-100, 100:-100] = 1

In [4]:
# input volume is vol (must be a box)

curr_width = vol.shape[0] # assumes cube
base_width = 64
points = [[0,0,0]]
vols = [vol]

while True:
    print(f"##{curr_width}")
    curr_width = int(np.ceil(curr_width/2))
    
    vols_new = []
    points_new = []
    for iv, point in enumerate(points):
        v, p = split_cube(vols[iv], point, curr_width)
        vols_new.append(v)
        points_new.append(p)
#     import pdb; pdb.set_trace()
    
    
    vols = np.asarray(vols_new).reshape((-1, curr_width, curr_width, curr_width))
    points = np.asarray(points_new).reshape((-1,3))
    
    print(curr_width)
    
    if curr_width == base_width:
        break

##512
256
##256


/home/atekawade/anaconda3/envs/tf2GPU/lib/python3.9/site-packages/numpy/core/_methods.py:261: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/atekawade/anaconda3/envs/tf2GPU/lib/python3.9/site-packages/numpy/core/_methods.py:221: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/home/atekawade/anaconda3/envs/tf2GPU/lib/python3.9/site-packages/numpy/core/_methods.py:250: RuntimeWarning: invalid value encountered in true_divide
  ret = um.true_divide(
/home/atekawade/anaconda3/envs/tf2GPU/lib/python3.9/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, o

ValueError: cannot reshape array of size 8 into shape (128,128,128)

In [ ]:
curr_width

In [ ]:
np.shape(vols_new)

In [ ]:
vols.shape

In [ ]:
view_midplanes(vol)